In [75]:
import pandas as pd
import altair as alt
import datetime as datetime
import pandas_datareader as DataReader
import scipy
import sklearn
from sklearn.linear_model import LinearRegression
import numpy as np

In [76]:
# Get the Bitcoin Historical Pricing Data
df = pd.read_csv("bitstampUSD.csv")
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# Problems with the data set
# NaN values early on, data is by the second which doesn't correspond
# with most technical indicators and other financial instruments
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4363457 entries, 0 to 4363456
Data columns (total 8 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Timestamp          int64  
 1   Open               float64
 2   High               float64
 3   Low                float64
 4   Close              float64
 5   Volume_(BTC)       float64
 6   Volume_(Currency)  float64
 7   Weighted_Price     float64
dtypes: float64(7), int64(1)
memory usage: 266.3 MB


In [78]:
df.describe()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
count,4.363457e+06,3.126480e+06,3.126480e+06,3.126480e+06,3.126480e+06,3.126480e+06,3.126480e+06,3.126480e+06
mean,1.456469e+09,3.674656e+03,3.677366e+03,3.671730e+03,3.674595e+03,9.855040e+00,2.884459e+04,3.674570e+03
std,7.573296e+07,3.935578e+03,3.939077e+03,3.931713e+03,3.935490e+03,3.229272e+01,1.010277e+05,3.935458e+03
min,1.325318e+09,3.800000e+00,3.800000e+00,1.500000e+00,1.500000e+00,0.000000e+00,0.000000e+00,3.800000e+00
25%,1.390770e+09,4.100000e+02,4.102400e+02,4.098300e+02,4.100000e+02,3.988120e-01,3.503759e+02,4.099998e+02
50%,1.456610e+09,1.175140e+03,1.175770e+03,1.174825e+03,1.175140e+03,1.990000e+00,2.620491e+03,1.175200e+03
75%,1.522062e+09,6.931175e+03,6.935780e+03,6.926790e+03,6.931225e+03,7.639098e+00,1.760057e+04,6.931180e+03
max,1.587514e+09,1.966576e+04,1.966600e+04,1.964996e+04,1.966575e+04,5.853852e+03,7.569437e+06,1.966330e+04


In [79]:
# what percent of set is null?
print(df.isnull().sum()*100/df.isnull().count())

Timestamp             0.000000
Open                 28.348555
High                 28.348555
Low                  28.348555
Close                28.348555
Volume_(BTC)         28.348555
Volume_(Currency)    28.348555
Weighted_Price       28.348555
dtype: float64


In [80]:
# we will ignore earilier dates with NaN values
bitcoin = df.dropna()
bitcoin

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000
...,...,...,...,...,...,...,...,...
4363452,1587513360,6847.97,6856.35,6847.97,6856.35,0.125174,858.128697,6855.498790
4363453,1587513420,6850.23,6856.13,6850.23,6850.89,1.224777,8396.781459,6855.763449
4363454,1587513480,6846.50,6857.45,6846.02,6857.45,7.089168,48533.089069,6846.090966
4363455,1587513540,6854.18,6854.98,6854.18,6854.98,0.012231,83.831604,6854.195090


In [81]:
# convert the timestamp variable to a datetime object
bitcoin['Timestamp']=pd.to_datetime(bitcoin['Timestamp'],unit='s')
# index by second
bitcoin.set_index('Timestamp', inplace = True)
# aggregate each group of seconds with by day for prediction, month for graphing
bitcoin_daily = bitcoin.resample('1D').mean()
bitcoin_monthly = bitcoin.resample('1M').mean()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [214]:
bitcoin_daily

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,day,S&P500,Nikkiei,Month,Year,Bad Guess
Timestamp,,,,,,,,,,,,,
2012-01-04,5.200000,5.223333,5.200000,5.223333,11.914807,63.119577,5.208159,2012-01-04,1277.30,8560.11,1,2012,False
2012-01-05,6.281429,6.286190,6.281429,6.286190,4.514373,27.987370,6.284127,2012-01-05,1281.06,8488.71,1,2012,False
2012-01-06,6.435000,6.445000,6.435000,6.445000,2.420196,15.914659,6.438999,2012-01-06,1277.81,8390.35,1,2012,False
2012-01-10,6.603000,6.603000,6.603000,6.603000,2.076333,13.773469,6.603000,2012-01-10,1292.08,8422.26,1,2012,False
2012-01-11,7.119583,7.121875,7.119583,7.121875,2.194978,15.479409,7.120685,2012-01-11,1292.48,8447.88,1,2012,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-04-16,6914.045839,6919.369179,6909.210340,6914.345924,11.130005,77496.548247,6914.666410,2020-04-16,2799.55,19290.20,4,2020,True
2020-04-17,7071.958939,7075.071649,7068.929333,7072.234602,6.121590,43287.908348,7071.901788,2020-04-17,2874.56,19897.26,4,2020,False
2020-04-20,7050.524520,7054.468401,7046.525416,7050.542267,8.077476,56419.883448,7050.526233,2020-04-20,2823.16,19669.12,4,2020,False


In [83]:
bitcoin_monthly

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,
2011-12-31,4.465000,4.482500,4.465000,4.482500,23.829470,106.330084,4.471603
2012-01-31,6.345389,6.348982,6.341218,6.346148,4.031777,25.168238,6.345955
2012-02-29,5.230208,5.231646,5.227036,5.228510,8.313993,42.239422,5.228443
2012-03-31,4.985481,4.986695,4.982580,4.983828,15.197791,76.509751,4.984397
2012-04-30,4.995171,4.996447,4.993763,4.995079,21.683913,108.218094,4.995091
...,...,...,...,...,...,...,...
2019-12-31,7240.451387,7244.225846,7236.783347,7240.441817,2.930000,21126.441888,7240.856314
2020-01-31,8324.445998,8329.409153,8320.019888,8324.543275,4.565741,38436.780169,8325.103326
2020-02-29,9646.123398,9653.150847,9640.673863,9646.404206,4.086245,39434.591482,9647.502249


In [84]:
# Set Index to column so it can be referenced in altair
bitcoin_monthly["Month"] = bitcoin_monthly.index
# Graph Bitcoin Price over time
alt.Chart(bitcoin_monthly).mark_line().encode(
    x = alt.X('Month', title='Year'),
    y = alt.Y('Close', axis=alt.Axis(format='$', title='Closing Price (USD)')),
    color=alt.value('orange'),
).properties(
    title='Bitcoin Historical Price'
).configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

alt.Chart(...)

Let's look at the general trend in Bitcoin Stock Prices over the past 8 years. There is a noticeable, meteoric rise in 2017 that is preceeded by years of slow growth.

In [216]:
# Do dates correspond with data?
bitcoin_daily["Month"] = pd.DatetimeIndex(bitcoin_daily['day']).month
bitcoin_daily["Year"] = pd.DatetimeIndex(bitcoin_daily['day']).year
bitcoin_daily.info

monthly_averages = bitcoin_daily.groupby('Month').mean()
monthly_averages["Month"] = monthly_averages.index
dates = {1:'January', 2:'February', 3 : 'March', 4: 'April', 5 : 'May', 6:'June', 7:'July', 8:'August', 9:'September', 10: 'October', 11: 'November', 12: 'December'}
monthly_averages.replace({"Month": dates}, inplace=True)

alt.Chart(monthly_averages).mark_bar().encode(
    x= alt.X('Month'),
    y= alt.Y('Close', axis=alt.Axis(format='$', title='"Average Price of Bitcoin in USD"'))
).properties(
    title = "Average Price of Bitcoin by Month"
).configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

alt.Chart(...)

SyntaxError: invalid syntax (<ipython-input-219-e1ecb746a741>, line 1)

In [215]:
# Often we use basic market indicators to predict stock performance
# Is Bitcoin Coorelated with other instruments as a whole
start = bitcoin_monthly["Month"].iloc[0]
end = bitcoin_monthly["Month"].iloc[100]
bitcoin_monthly["S&P500"] = DataReader.DataReader(['sp500'], 'fred', start, end)
bitcoin_monthly["Nikkiei"] = DataReader.DataReader(['NIKKEI225'], 'fred', start, end)
#bitcoin_monthly["GDP"] = DataReader.DataReader('A191RL1Q225SBEA', 'fred', start, end)
bitcoin_monthly

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Month,S&P500,Nikkiei
Timestamp,,,,,,,,,,
2011-12-31,4.465000,4.482500,4.465000,4.482500,23.829470,106.330084,4.471603,2011-12-31,NaN,NaN
2012-01-31,6.345389,6.348982,6.341218,6.346148,4.031777,25.168238,6.345955,2012-01-31,1312.41,8802.51
2012-02-29,5.230208,5.231646,5.227036,5.228510,8.313993,42.239422,5.228443,2012-02-29,1365.68,9723.24
2012-03-31,4.985481,4.986695,4.982580,4.983828,15.197791,76.509751,4.984397,2012-03-31,NaN,NaN
2012-04-30,4.995171,4.996447,4.993763,4.995079,21.683913,108.218094,4.995091,2012-04-30,1397.91,NaN
...,...,...,...,...,...,...,...,...,...,...
2019-12-31,7240.451387,7244.225846,7236.783347,7240.441817,2.930000,21126.441888,7240.856314,2019-12-31,3230.78,NaN
2020-01-31,8324.445998,8329.409153,8320.019888,8324.543275,4.565741,38436.780169,8325.103326,2020-01-31,3225.52,23205.18
2020-02-29,9646.123398,9653.150847,9640.673863,9646.404206,4.086245,39434.591482,9647.502249,2020-02-29,NaN,NaN


In [86]:
# is bitcoin coorelated with these indicators
bitcoin_monthly.corr()['Close'].iloc[7:]

S&P500     0.841347
Nikkiei    0.705686
Name: Close, dtype: float64

In [222]:
# Nicely visualized with chart below
bitcoin_sp = alt.Chart(bitcoin_monthly).mark_point().encode(
    x = alt.X('Close', axis=alt.Axis(format='$', title='Bitcoin (USD)')),
    y = alt.Y('S&P500',axis=alt.Axis(format='$', title='S&P (USD)')),
    color=alt.value('green'),
).mark_circle(size=30)
(bitcoin_sp + bitcoin_sp.transform_regression('Close', 'S&P500').mark_line()).configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
).properties(
    title = "Coorelation Between Bitcoin, S&P 500"
)

alt.LayerChart(...)

In [88]:
bitcoin_sp = alt.Chart(bitcoin_monthly).mark_point().encode(
    x = alt.X('Close', axis=alt.Axis(format='$', title='Bitcoin (USD)')),
    y = alt.Y('Nikkiei',axis=alt.Axis(format='$', title='Nikkiei (Tokyo) (USD)')),
    color=alt.value('red'),
).mark_circle(size=30)
(bitcoin_sp + bitcoin_sp.transform_regression('Close', 'Nikkiei').mark_line()).configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
).properties(
    title = "Coorelation Between Bitcoin, Nikkie Index"
)

alt.LayerChart(...)

In [89]:
# It seems like it is, so lets fit a model with these values
# first add data to day by day value
bitcoin_daily["day"] = bitcoin_daily.index


start = bitcoin_daily["day"].iloc[0]
end = bitcoin_daily["day"].iloc[3035]
bitcoin_daily["S&P500"] = DataReader.DataReader(['sp500'], 'fred', start, end)
bitcoin_daily["Nikkiei"] = DataReader.DataReader(['NIKKEI225'], 'fred', start, end)

In [90]:
print(bitcoin_daily.isnull().sum()*100/bitcoin_daily.isnull().count())

Open                  0.098814
High                  0.098814
Low                   0.098814
Close                 0.098814
Volume_(BTC)          0.098814
Volume_(Currency)     0.098814
Weighted_Price        0.098814
day                   0.000000
S&P500               31.192358
Nikkiei              33.036891
dtype: float64


In [91]:
# NaNs correspond to weekends/holidays, so we drop them
bitcoin_daily.dropna(inplace = True)

In [92]:
bitcoin_daily

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,day,S&P500,Nikkiei
Timestamp,,,,,,,,,,
2012-01-04,5.200000,5.223333,5.200000,5.223333,11.914807,63.119577,5.208159,2012-01-04,1277.30,8560.11
2012-01-05,6.281429,6.286190,6.281429,6.286190,4.514373,27.987370,6.284127,2012-01-05,1281.06,8488.71
2012-01-06,6.435000,6.445000,6.435000,6.445000,2.420196,15.914659,6.438999,2012-01-06,1277.81,8390.35
2012-01-10,6.603000,6.603000,6.603000,6.603000,2.076333,13.773469,6.603000,2012-01-10,1292.08,8422.26
2012-01-11,7.119583,7.121875,7.119583,7.121875,2.194978,15.479409,7.120685,2012-01-11,1292.48,8447.88
...,...,...,...,...,...,...,...,...,...,...
2020-04-16,6914.045839,6919.369179,6909.210340,6914.345924,11.130005,77496.548247,6914.666410,2020-04-16,2799.55,19290.20
2020-04-17,7071.958939,7075.071649,7068.929333,7072.234602,6.121590,43287.908348,7071.901788,2020-04-17,2874.56,19897.26
2020-04-20,7050.524520,7054.468401,7046.525416,7050.542267,8.077476,56419.883448,7050.526233,2020-04-20,2823.16,19669.12


In [93]:
# Do dates correspond with data?
bitcoin_daily["Month"] = pd.DatetimeIndex(bitcoin_daily['day']).month
bitcoin_daily["Year"] = pd.DatetimeIndex(bitcoin_daily['day']).year
bitcoin_daily.info

<bound method DataFrame.info of                    Open         High          Low        Close  Volume_(BTC)  \
Timestamp                                                                      
2012-01-04     5.200000     5.223333     5.200000     5.223333     11.914807   
2012-01-05     6.281429     6.286190     6.281429     6.286190      4.514373   
2012-01-06     6.435000     6.445000     6.435000     6.445000      2.420196   
2012-01-10     6.603000     6.603000     6.603000     6.603000      2.076333   
2012-01-11     7.119583     7.121875     7.119583     7.121875      2.194978   
...                 ...          ...          ...          ...           ...   
2020-04-16  6914.045839  6919.369179  6909.210340  6914.345924     11.130005   
2020-04-17  7071.958939  7075.071649  7068.929333  7072.234602      6.121590   
2020-04-20  7050.524520  7054.468401  7046.525416  7050.542267      8.077476   
2020-04-21  6867.094397  6869.864491  6864.215868  6867.164541      3.996382   
2020-04-

In [94]:
monthly_averages = bitcoin_daily.groupby('Month').mean()
monthly_averages["Month"] = monthly_averages.index
dates = {1:'January', 2:'February', 3 : 'March', 4: 'April', 5 : 'May', 6:'June', 7:'July', 8:'August', 9:'September', 10: 'October', 11: 'November', 12: 'December'}
monthly_averages.replace({"Month": dates}, inplace=True)

In [95]:
alt.Chart(monthly_averages).mark_bar().encode(
    x= alt.X('Month'),
    y= alt.Y('Close', axis=alt.Axis(format='$', title='"Average Price of Bitcoin in USD"'))
).properties(
    title = "Average Price of Bitcoin by Month"
).configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

alt.Chart(...)

In [121]:
# Make a model
def add_bias(data):
    data.insert(0, 'ones', np.ones(len(data)))

Y = bitcoin_daily["Close"]
X = bitcoin_daily[["Volume_(BTC)", "Volume_(Currency)", "S&P500", "Nikkiei", "Month"]]

In [122]:
add_bias(X)

In [123]:
X.replace({"Month": dates}, inplace=True)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:6666: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regex=regex,


In [124]:
X

,ones,Volume_(BTC),Volume_(Currency),S&P500,Nikkiei,Month
Timestamp,,,,,,
2012-01-04,1.0,11.914807,63.119577,1277.30,8560.11,January
2012-01-05,1.0,4.514373,27.987370,1281.06,8488.71,January
2012-01-06,1.0,2.420196,15.914659,1277.81,8390.35,January
2012-01-10,1.0,2.076333,13.773469,1292.08,8422.26,January
2012-01-11,1.0,2.194978,15.479409,1292.48,8447.88,January
...,...,...,...,...,...,...
2020-04-16,1.0,11.130005,77496.548247,2799.55,19290.20,April
2020-04-17,1.0,6.121590,43287.908348,2874.56,19897.26,April
2020-04-20,1.0,8.077476,56419.883448,2823.16,19669.12,April


In [125]:
# One Hot Encoding
from sklearn.feature_extraction import DictVectorizer

# one-hot encoding 
X_features_with_Month=X

Month = X_features_with_Month[['Month']].to_dict(orient='records') 

encoder = DictVectorizer(sparse=False)
Month_df = pd.DataFrame(
    data = encoder.fit_transform(Month),
    columns = encoder.feature_names_
)


# adjusting the index inconsistency issue
X_features_with_Month.reset_index(drop=True, inplace=True)
Month_df.reset_index(drop=True, inplace=True)

# Combine the features together with pd.concat
X_with_Month = pd.concat([X_features_with_Month,Month_df],ignore_index=False,axis=1).drop(['Month'],axis=1)
X_with_Month

,ones,Volume_(BTC),Volume_(Currency),S&P500,Nikkiei,Month=April,Month=August,Month=December,Month=February,Month=January,Month=July,Month=June,Month=March,Month=May,Month=November,Month=October,Month=September
0,1.0,11.914807,63.119577,1277.30,8560.11,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,4.514373,27.987370,1281.06,8488.71,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,2.420196,15.914659,1277.81,8390.35,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,2.076333,13.773469,1292.08,8422.26,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,2.194978,15.479409,1292.48,8447.88,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1956,1.0,11.130005,77496.548247,2799.55,19290.20,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1957,1.0,6.121590,43287.908348,2874.56,19897.26,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1958,1.0,8.077476,56419.883448,2823.16,19669.12,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1959,1.0,3.996382,27413.523629,2736.56,19280.78,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [138]:
def avg_squared_loss(y, y_hat):
    #return np.array([(y[i] - y_hat[i])**2 for i in range(0,len(y))]).sum()/len(y)
    return np.mean((y - y_hat)**2)

model = LinearRegression()
result = model.fit(X_with_Month,Y)
Y_hat = model.predict(X_with_Month)
loss = avg_squared_loss(Y_hat, Y)
loss

1761473.151427865

In [139]:
model.coef_

array([ 0.00000000e+00, -2.45933617e+01,  4.26158277e-02,  6.90674317e+00,
       -4.12876292e-01, -2.17648235e+02,  9.47761973e+01,  2.58093071e+02,
       -2.75450054e+02,  1.59654050e+02, -3.39106370e+01, -1.56598246e+01,
       -2.88596006e+02,  6.41257203e+01,  1.25803062e+02,  1.78010233e+02,
       -4.91975769e+01])

In [140]:
model.coef_.max()

258.093070856881

In [209]:
alt.data_transformers.disable_max_rows()

source = pd.DataFrame({
    'Y': Y,
    'Y_hat': Y_hat
})

layer1 = alt.Chart(source).mark_circle(size=4).encode(
    x='Y',
    y='Y_hat',
    color = alt.value('red')
).properties(
    title='Expected (Y) vs Prediction (Y_Hat)'
)

layer1 + layer1.transform_regression('Y', 'Y_hat').mark_line()

alt.LayerChart(...)

In [159]:
# Find worse predictions of Linear Model SCIKIT Learn
bad_predictions = pd.DataFrame(data=Y, columns=["Y"])
bad_predictions["Predicted"] = Y
bad_predictions["Y_hat"] = Y_hat
bad_predictions["Difference"] = (bad_predictions["Predicted"] - bad_predictions["Y_hat"])**2

In [160]:
# Tends to be inaccurate for larger values of Y

In [176]:
bad_predictions["Bad Guess"] = bad_predictions["Difference"] > bad_predictions["Difference"].mean()

In [177]:
bad_predictions[bad_predictions["Bad Guess"]==True]
bitcoin_daily["Bad Guess"] = bad_predictions["Bad Guess"]

In [178]:
bad_dates = bitcoin_daily[bitcoin_daily["Bad Guess"]==True].groupby('Month').count()
bad_dates["Month"] = bad_dates.index
dates = {1:'January', 2:'February', 3 : 'March', 4: 'April', 5 : 'May', 6:'June', 7:'July', 8:'August', 9:'September', 10: 'October', 11: 'November', 12: 'December'}
bad_dates.replace({"Month": dates}, inplace=True)

In [179]:
alt.Chart(bad_dates).mark_bar().encode(
    x= alt.X('Month'),
    y= alt.Y('Close', axis=alt.Axis(title='"Count of Bad Predictions"'))
).properties(
    title = "Bad Predictions by Date"
).configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

alt.Chart(...)

In [184]:
bitcoin_daily.loc[bad_predictions["Difference"].idxmax()]

Open                             13759.1
High                             13807.6
Low                                13707
Close                            13758.1
Volume_(BTC)                     40.1712
Volume_(Currency)                 539960
Weighted_Price                   13756.4
day                  2017-12-22 00:00:00
S&P500                           2683.34
Nikkiei                          22902.8
Month                                 12
Year                                2017
Bad Guess                           True
Name: 2017-12-22 00:00:00, dtype: object

In [181]:
bad_predictions

,Y,Predicted,Y_hat,Difference,Bad Guess
Timestamp,,,,,
2012-01-04,NaN,5.223333,-692.585665,4.869374e+05,False
2012-01-05,NaN,6.286190,-456.632585,2.142938e+05,False
2012-01-06,NaN,6.445000,-387.480634,1.551774e+05,False
2012-01-10,NaN,6.603000,-293.730780,9.020038e+04,False
2012-01-11,NaN,7.121875,-304.391155,9.704037e+04,False
...,...,...,...,...,...
2020-04-16,NaN,6914.345924,8332.892647,2.012275e+06,True
2020-04-17,NaN,7072.234602,7265.671012,3.741764e+04,False
2020-04-20,NaN,7050.542267,7516.386181,2.170106e+05,False


In [212]:
bad_dates2 = bitcoin_daily[bitcoin_daily["Bad Guess"]==True].groupby('Year').count()
bad_dates2["Year"] = bad_dates2.index
#dates = {1:'January', 2:'February', 3 : 'March', 4: 'April', 5 : 'May', 6:'June', 7:'July', 8:'August', 9:'September', 10: 'October', 11: 'November', 12: 'December'}
#bad_dates.replace({"Month": dates}, inplace=True)

In [226]:
alt.Chart(bad_dates2).mark_bar(size=10).encode(
    x= alt.X('Year:O'),
    y= alt.Y('Close', axis=alt.Axis(title='Count of Bad Predictions'))
).properties(
    title = "Bad Predictions by Year"
).configure_title(
    fontSize=20,
    font='Courier',
    anchor='start',
    color='gray'
)

alt.Chart(...)

In [ ]:
# Notice that most of the changes took place during bitcoins meteoric rise, when the predicitons were lower.